# Eye Color Prediction Parte 3: Criação do Modelo

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import OneHotEncoder


from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

## Setup Inicial

In [2]:
df = pd.read_csv('datasets/df_ml_clean_2021.csv')

In [3]:
df.head(2)

,rs1129038,rs11636232,rs12203592,rs12896399,rs12913832,rs1393350,rs1667394,rs16891982,rs1800407,rs4778232,rs4778241,rs7183877,rs8024968,color_cat
0,TC,CC,missing,missing,missing,missing,missing,missing,CC,TC,CA,CC,CC,Brown
1,TT,TC,missing,missing,missing,missing,missing,missing,TC,CC,CA,CC,CC,BGG


In [4]:
df.shape

(1231, 14)

In [5]:
# Conjunto com as SNPs
snp_list_6 = ['rs12913832',
    'rs1800407',
    'rs12896399',
    'rs16891982',
    'rs1393350',
    'rs12203592']

snp_list_13 = [
    'rs1129038', 
    'rs11636232', 
    'rs12203592',
    'rs12896399',
    'rs12913832', 
    'rs1393350',
    'rs1667394',
    'rs16891982',
    'rs1800407',
    'rs4778232',
    'rs4778241',
    'rs7183877',
    'rs8024968']
snp_list_remove = list(set(snp_list_13) - set(snp_list_6))

In [6]:
df = df.drop(columns=snp_list_remove)

In [7]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [8]:
X_train.nunique()

rs12203592    4
rs12896399    4
rs12913832    4
rs1393350     4
rs16891982    4
rs1800407     4
dtype: int64

In [9]:
# encoding
ohe = OneHotEncoder(sparse=False)
ohe.fit(X_train)

X_train_encoded = ohe.transform(X_train)
X_test_encoded = ohe.transform(X_test)

/Users/emanuele/code/Phenotype_Prediction/env/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


### Experimento 01: Regressão Logística

In [10]:
space01 = {"C":np.logspace(-4, 4, 50),
            "fit_intercept":[True,False]
            }

model01 = LogisticRegression(multi_class='multinomial', max_iter=1000 )
cv01 = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
search01 = GridSearchCV(model01, space01, cv=cv01, n_jobs=-1, verbose=10)
result01 = search01.fit(X_train_encoded, y_train)

Fitting 15 folds for each of 100 candidates, totalling 1500 fits
[CV 8/15; 1/100] START C=0.0001, fit_intercept=True.............................
[CV 7/15; 1/100] START C=0.0001, fit_intercept=True.............................
[CV 4/15; 1/100] START C=0.0001, fit_intercept=True.............................
[CV 8/15; 1/100] END C=0.0001, fit_intercept=True;, score=0.477 total time=   0.0s
[CV 7/15; 1/100] END C=0.0001, fit_intercept=True;, score=0.447 total time=   0.0s
[CV 4/15; 1/100] END C=0.0001, fit_intercept=True;, score=0.406 total time=   0.0s
[CV 5/15; 1/100] START C=0.0001, fit_intercept=True.............................
[CV 3/15; 1/100] START C=0.0001, fit_intercept=True.............................
[CV 1/15; 1/100] START C=0.0001, fit_intercept=True.............................
[CV 6/15; 1/100] START C=0.0001, fit_intercept=True.............................
[CV 2/15; 1/100] START C=0.0001, fit_intercept=True.............................
[CV 5/15; 1/100] END C=0.0001, fit_int

In [11]:
print(f"Os melhores parâmetros encontrados foram: {result01.best_params_}")
best_model01 = result01.best_estimator_

y_pred = best_model01.predict(X_test_encoded)
y_pred_probs = best_model01.predict_proba(X_test_encoded)
y_pred_train = best_model01.predict(X_train_encoded)

print(f"Roc auc score do modelo: {roc_auc_score(y_test, y_pred_probs, average='weighted', multi_class='ovr') }")
print(f"Acurácia de teste: {accuracy_score(y_test, y_pred):.2%}.")
print(f"Acurácia de treino: { accuracy_score(y_pred_train, y_train):.2%}")
print(" \n Classification Report: \n ")
print(classification_report(y_test, y_pred, target_names=y.unique()))

Os melhores parâmetros encontrados foram: {'C': 0.5689866029018293, 'fit_intercept': False}
Roc auc score do modelo: 0.8608404094728358
Acurácia de teste: 71.26%.
Acurácia de treino: 70.33%
 
 Classification Report: 
 
              precision    recall  f1-score   support

       Brown       0.76      0.85      0.81       107
         BGG       0.64      0.91      0.75        75
         Int       0.81      0.26      0.40        65

    accuracy                           0.71       247
   macro avg       0.74      0.67      0.65       247
weighted avg       0.74      0.71      0.68       247



### Experimento 02: Random Forest

In [12]:
n_estimators = [120, 300, 500, 800, 1200]
max_features = ['auto', 'sqrt','log2']
max_depth = [5, 8, 15, 25, 30, None]
min_samples_split = [1, 2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10]

space02 = {'n_estimators': n_estimators,
           'max_features': max_features,
           'max_depth': max_depth,
           'min_samples_split': min_samples_split,
           'min_samples_leaf': min_samples_leaf,
           } 

model02 = RandomForestClassifier(random_state=42)
cv02 = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
search02 = RandomizedSearchCV(model02, space02, cv=cv02, n_jobs=-1, n_iter=50, verbose=10)
result02 = search02.fit(X_train_encoded, y_train)

Fitting 15 folds for each of 50 candidates, totalling 750 fits
[CV 3/15; 1/50] START max_depth=25, max_features=log2, min_samples_leaf=5, min_samples_split=5, n_estimators=1200
[CV 1/15; 1/50] START max_depth=25, max_features=log2, min_samples_leaf=5, min_samples_split=5, n_estimators=1200
[CV 2/15; 1/50] START max_depth=25, max_features=log2, min_samples_leaf=5, min_samples_split=5, n_estimators=1200
[CV 4/15; 1/50] START max_depth=25, max_features=log2, min_samples_leaf=5, min_samples_split=5, n_estimators=1200
[CV 5/15; 1/50] START max_depth=25, max_features=log2, min_samples_leaf=5, min_samples_split=5, n_estimators=1200
[CV 6/15; 1/50] START max_depth=25, max_features=log2, min_samples_leaf=5, min_samples_split=5, n_estimators=1200
[CV 8/15; 1/50] START max_depth=25, max_features=log2, min_samples_leaf=5, min_samples_split=5, n_estimators=1200
[CV 7/15; 1/50] START max_depth=25, max_features=log2, min_samples_leaf=5, min_samples_split=5, n_estimators=1200
[CV 2/15; 1/50] END max_d

/Users/emanuele/code/Phenotype_Prediction/env/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV 1/15; 2/50] END max_depth=25, max_features=auto, min_samples_leaf=5, min_samples_split=10, n_estimators=500;, score=0.701 total time=   2.0s
[CV 2/15; 2/50] START max_depth=25, max_features=auto, min_samples_leaf=5, min_samples_split=10, n_estimators=500


/Users/emanuele/code/Phenotype_Prediction/env/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV 2/15; 2/50] END max_depth=25, max_features=auto, min_samples_leaf=5, min_samples_split=10, n_estimators=500;, score=0.685 total time=   2.3s
[CV 3/15; 2/50] START max_depth=25, max_features=auto, min_samples_leaf=5, min_samples_split=10, n_estimators=500


...

/Users/emanuele/code/Phenotype_Prediction/env/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/emanuele/code/Phenotype_Prediction/env/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/emanuele/code/Phenotype_Prediction/env/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, ex

[CV 6/15; 46/50] END max_depth=25, max_features=auto, min_samples_leaf=10, min_samples_split=15, n_estimators=120;, score=0.716 total time=   0.6s
[CV 13/15; 46/50] START max_depth=25, max_features=auto, min_samples_leaf=10, min_samples_split=15, n_estimators=120
[CV 5/15; 46/50] END max_depth=25, max_features=auto, min_samples_leaf=10, min_samples_split=15, n_estimators=120;, score=0.663 total time=   0.7s
[CV 14/15; 46/50] START max_depth=25, max_features=auto, min_samples_leaf=10, min_samples_split=15, n_estimators=120
[CV 7/15; 46/50] END max_depth=25, max_features=auto, min_samples_leaf=10, min_samples_split=15, n_estimators=120;, score=0.721 total time=   0.7s
[CV 15/15; 46/50] START max_depth=25, max_features=auto, min_samples_leaf=10, min_samples_split=15, n_estimators=120


/Users/emanuele/code/Phenotype_Prediction/env/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/emanuele/code/Phenotype_Prediction/env/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/emanuele/code/Phenotype_Prediction/env/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, ex

[CV 8/15; 46/50] END max_depth=25, max_features=auto, min_samples_leaf=10, min_samples_split=15, n_estimators=120;, score=0.604 total time=   0.7s
[CV 1/15; 47/50] START max_depth=5, max_features=log2, min_samples_leaf=5, min_samples_split=1, n_estimators=1200
[CV 9/15; 46/50] END max_depth=25, max_features=auto, min_samples_leaf=10, min_samples_split=15, n_estimators=120;, score=0.680 total time=   0.7s
[CV 2/15; 47/50] START max_depth=5, max_features=log2, min_samples_leaf=5, min_samples_split=1, n_estimators=1200
[CV 10/15; 46/50] END max_depth=25, max_features=auto, min_samples_leaf=10, min_samples_split=15, n_estimators=120;, score=0.689 total time=   0.8s
[CV 3/15; 47/50] START max_depth=5, max_features=log2, min_samples_leaf=5, min_samples_split=1, n_estimators=1200
[CV 11/15; 46/50] END max_depth=25, max_features=auto, min_samples_leaf=10, min_samples_split=15, n_estimators=120;, score=0.690 total time=   0.8s
[CV 4/15; 47/50] START max_depth=5, max_features=log2, min_samples_l

/Users/emanuele/code/Phenotype_Prediction/env/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


In [13]:
print(f"Os melhores parâmetros encontrados foram: {result02.best_params_}")
best_model02 = result02.best_estimator_

y_pred = best_model02.predict(X_test_encoded)
y_pred_probs = best_model02.predict_proba(X_test_encoded)
y_pred_train = best_model02.predict(X_train_encoded)

print(f"Roc auc score do modelo: {roc_auc_score(y_test, y_pred_probs, average='weighted', multi_class='ovr') }")
print(f"Acurácia de teste: {accuracy_score(y_test, y_pred):.2%}.")
print(f"Acurácia de treino: { accuracy_score(y_pred_train, y_train):.2%}")
print(" \n Classification Report: \n ")
print(classification_report(y_test, y_pred, target_names=y.unique()))

Os melhores parâmetros encontrados foram: {'n_estimators': 120, 'min_samples_split': 1, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 5}
Roc auc score do modelo: 0.8597884814554619
Acurácia de teste: 71.66%.
Acurácia de treino: 70.53%
 
 Classification Report: 
 
              precision    recall  f1-score   support

       Brown       0.81      0.81      0.81       107
         BGG       0.62      0.99      0.76        75
         Int       0.80      0.25      0.38        65

    accuracy                           0.72       247
   macro avg       0.74      0.68      0.65       247
weighted avg       0.75      0.72      0.68       247



### Experimento 03: XGBoosting

In [14]:
y_train_map = y_train.replace({"BGG":0, "Brown":1, "Int":2})
y_test_map = y_test.replace({"BGG":0, "Brown":1, "Int":2})


space03 = {'eta': [0.01, 0.015, 0.025, 0.05, 0.1],
           'gamma': [0.05,0.01,0.3,0.5,0.7,0.9,1.0],
           'max_depth': [3,5,7,9,12,15,17,25],
           'min_child_weight': [1,3,5,7],
           'subsample': [0.6,0.7,0.8,0.9,1.0],
           'colsample_bytree':[0.6,0.7,0.8,0.9,1.0],
           'lambda':[0.01,0.1,1.0],
           'alpha':[0,0.1,0.5,1.0],
           } 

model03 = xgb.XGBClassifier(objective="multi:softproba", num_class=3, random_state=42, verbosity=1)
cv03 = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
search03 = RandomizedSearchCV(model03, space03, cv=cv03, n_jobs=-1, n_iter=350, verbose=10)
result03 = search03.fit(X_train_encoded, y_train_map)

Fitting 15 folds for each of 350 candidates, totalling 5250 fits
[CV 1/15; 1/350] START alpha=0, colsample_bytree=0.6, eta=0.01, gamma=0.05, lambda=0.01, max_depth=17, min_child_weight=1, subsample=0.9
[CV 2/15; 1/350] START alpha=0, colsample_bytree=0.6, eta=0.01, gamma=0.05, lambda=0.01, max_depth=17, min_child_weight=1, subsample=0.9
[CV 3/15; 1/350] START alpha=0, colsample_bytree=0.6, eta=0.01, gamma=0.05, lambda=0.01, max_depth=17, min_child_weight=1, subsample=0.9
[CV 4/15; 1/350] START alpha=0, colsample_bytree=0.6, eta=0.01, gamma=0.05, lambda=0.01, max_depth=17, min_child_weight=1, subsample=0.9
[CV 5/15; 1/350] START alpha=0, colsample_bytree=0.6, eta=0.01, gamma=0.05, lambda=0.01, max_depth=17, min_child_weight=1, subsample=0.9
[CV 6/15; 1/350] START alpha=0, colsample_bytree=0.6, eta=0.01, gamma=0.05, lambda=0.01, max_depth=17, min_child_weight=1, subsample=0.9
[CV 7/15; 1/350] START alpha=0, colsample_bytree=0.6, eta=0.01, gamma=0.05, lambda=0.01, max_depth=17, min_child_

In [15]:
print(f"Os melhores parâmetros encontrados foram: {result03.best_params_} \n")
best_model03 = result03.best_estimator_

y_pred = best_model03.predict(X_test_encoded)
y_pred_probs = best_model03.predict_proba(X_test_encoded)
y_pred_train = best_model03.predict(X_train_encoded)

print(f"Roc auc score do modelo: {roc_auc_score(y_test_map, y_pred_probs, average='weighted', multi_class='ovr') }")
print(f"Acurácia de teste: {accuracy_score(y_test_map, y_pred):.2%}.")
print(f"Acurácia de treino: { accuracy_score(y_pred_train, y_train_map):.2%}")
print(" \n Classification Report: \n ")
print(classification_report(y_test_map, y_pred, target_names=y.unique()))

Os melhores parâmetros encontrados foram: {'subsample': 0.9, 'min_child_weight': 1, 'max_depth': 12, 'lambda': 0.01, 'gamma': 0.5, 'eta': 0.01, 'colsample_bytree': 0.6, 'alpha': 1.0} 

Roc auc score do modelo: 0.8613036219970595
Acurácia de teste: 72.47%.
Acurácia de treino: 70.63%
 
 Classification Report: 
 
              precision    recall  f1-score   support

       Brown       0.81      0.80      0.81       107
         BGG       0.63      1.00      0.77        75
         Int       0.82      0.28      0.41        65

    accuracy                           0.72       247
   macro avg       0.75      0.69      0.66       247
weighted avg       0.76      0.72      0.69       247

